In [1]:
from langchain_core.output_parsers import JsonOutputParser
from App.llms import model, ollama_model
from App.prompts import query_refinement, image_query_extraction, products_choice
from langchain_core.prompts import ChatPromptTemplate
import base64
from langchain_core.messages import HumanMessage
from qdrant_client import QdrantClient,models
from App.Hybrid_Search import HybridSearcher

client = QdrantClient(url="http://localhost:6333")

class Pipeline :
    def __init__(self):
        self.hybrid_searcher = HybridSearcher(collection_name="products")
        self.prompt_refinement = ChatPromptTemplate.from_template(query_refinement)
        self.prompt_choice = ChatPromptTemplate.from_template(products_choice)
        self.chain_refinement = self.prompt_refinement | model | JsonOutputParser()
        self.chain_choice = self.prompt_choice | model

    def describe_image(self, image_path: str):
        with open(image_path, "rb") as image_file:
            image_data = base64.b64encode(image_file.read()).decode("utf-8")

        message = HumanMessage(
            content=[
                {"type": "text", "text": image_query_extraction},
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
                },
            ]
        )

        try:
            response = model.invoke([message])
            return response.content
        except Exception as e:
            return ""

    def search (self, query,filters):
        return self.hybrid_searcher.search(query, filters)

    def refine_query(self,query):
        try:
            answer = self.chain_refinement.invoke({"query": query})
            return answer
        except Exception as e:
            return {"error": f"Failed to parse: {str(e)}", "raw_query": query}


    def make_choice(self,query,product_list):
        try:
            answer = self.chain_choice.invoke({"query": query, "product_list": product_list})
            return answer.content
        except Exception as e:
            return {"error": f"Failed to rerank products : {str(e)}", "raw_query": query}

    def pipeline(self,query:str,image_path:str=None):
        if(image_path):
            try:
                query += self.describe_image(image_path)
                print("image described")
            except Exception as e:
                print("NAAAAAAAAAAAAAAAAAAAAAAAAH")

        print("hedhi l query dyelna : " + query)
        refined_query = self.refine_query(query)
        print("hedhi l query dyelna kifeh wallet : " ,refined_query)
        query_filter = models.Filter(
            should=[
                models.FieldCondition(
                    key="discounted_price",
                    range=models.Range(lte=refined_query["filters"]["max_price"]),
                )
            ],
        )
        preliminary_results = self.search(query,query_filter)
        print(preliminary_results)
        result = self.make_choice(query,preliminary_results)
        return result

C:\Users\GIGABYTE\PycharmProjects\VectorsHackthon\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
query = "a good affordable camera"
image_path = "camera.jpg"
pipeline_rag = Pipeline()
res = pipeline_rag.pipeline(query = query, image_path = image_path)

image described
hedhi l query dyelna : a good affordable cameraA classic Nikon film SLR camera, featuring a silver-toned metal body with a black textured grip, fitted with a visible Nikon Series E lens. This professional-grade camera is revered by enthusiasts for its durable, high-quality design.
hedhi l query dyelna kifeh wallet :  {'semantic_query': 'Nikon film SLR camera Camera A classic Nikon film SLR camera, featuring a silver-toned metal body with a black textured grip, fitted with a visible Nikon Series E lens. This professional-grade camera is revered by enthusiasts for its durable, high-quality design.', 'filters': {'max_price': 350, 'monthly_cash_flow': 0, 'category': 'Camera'}, 'keywords': ['Nikon', 'film', 'SLR', 'camera', 'classic', 'silver', 'metal', 'black', 'grip', 'Series E lens', 'professional', 'durable', 'high-quality', 'affordable']}
[{'description': "product : Digitek DTR 550 LW (67 Inch) Tripod For DSLR, Camera |Operating Height: 5.57 Feet | Maximum Load Capacity

In [3]:
res

'```json\n[\n  {\n    "product_name": "Digitek DTR 550 LW (67 Inch) Tripod For DSLR, Camera",\n    "reason_for_being_fit": "While not a camera itself, the Digitek DTR 550 LW Tripod is a crucial accessory for achieving high-quality photography, aligning with the user\'s desire for a \'good camera\' experience. At a significantly discounted price of 1.549, it offers excellent affordability. Its features, such as a professional-grade adjustable height, multi-level locking, and a 360-degree ball head, enable stable and precise shots, which are essential for producing professional-looking images and videos. This investment enhances the capability of any camera, making it a financially smart choice for someone looking to elevate their photographic output on a budget, directly contributing to the \'good\' aspect of a camera setup."\n  }\n]\n```'